In [35]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [36]:
import pandas as pd
import re
import numpy as np
from sklearn import metrics


In [37]:
path = './result'
file_list = os.listdir(path)

In [ ]:

def sc_calc_acc_condition_with_temp_with_sc(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temp, excel_ver):
    # ./result/sc_l_result_4_15_Y_30_sys_prompt8_0.01_ver1_0.csv
    # print(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')
    tmp = pd.DataFrame()
    df_eval = pd.DataFrame()
    acc_list = []
    df = pd.DataFrame()

    file_list = os.listdir(path)
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')]
    opt_file = [x for x in opt_file if x.endswith(f'.csv')]

    if len(opt_file)>0 : 
        print(f'완료된 파일 개수 : {len(opt_file)}')
        for f in opt_file:
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            tmp = tmp.dropna()

            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp = tmp[tmp['o_result'].isin(['1', '0', '2'])]

            
            gold_df = tmp[['id', 'gold']].drop_duplicates()
            chk_cnt = tmp.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'question']]
            chk_cnt = chk_cnt.rename(columns = {'question': 'cnt'})
            chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]
            chk_cnt = chk_cnt.sort_values(by = ['id', 'cnt'], ascending=[True, False]).groupby(['id']).head(1)
            df_eval = pd.merge(gold_df, chk_cnt, on = ['id'])

            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            acc_list.append(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        y_true = df['o_result']
        y_pred = df['gold']
        print(metrics.classification_report(y_true, y_pred, digits=3))

        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [39]:
# process3 = Process(target=task, args=('v', 4, 10, 'Y', 10, 'sys_prompt10', 5, 0.01, 'ver5'))
sc_calc_acc_condition_with_temp_with_sc('v', 4, 10, 'Y', 10, 'sys_prompt10', 5,  0.01, 'ver5')

              precision    recall  f1-score   support

           0      1.000     0.889     0.941        18
           1      0.875     1.000     0.933        14
           2      1.000     1.000     1.000         2

    accuracy                          0.941        34
   macro avg      0.958     0.963     0.958        34
weighted avg      0.949     0.941     0.941        34

v_result_4_10_Y :  94.11764705882352


In [40]:
# process3 = Process(target=task, args=('v', 4, 10, 'Y', 10, 'sys_prompt10', 5, 0.01, 'ver5'))
# ('v', 4, 30, 'Y', 100, 'sys_prompt10', 5, 0.01, 'ver5')
sc_calc_acc_condition_with_temp_with_sc('v', 4, 30, 'Y', 100, 'sys_prompt10', 5,  0.01, 'ver5')

              precision    recall  f1-score   support

           0      0.868     0.781     0.822       320
           1      0.720     0.862     0.785       298
           2      0.938     0.659     0.774        91

    accuracy                          0.800       709
   macro avg      0.842     0.768     0.794       709
weighted avg      0.815     0.800     0.800       709

v_result_4_30_Y :  79.97179125528913
